In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")
import gcsfs
from pandas.io.formats.format import return_docstring
import re

In [ ]:
!pip install pandas gcsfs fastparquet

### Creación de los df a partir de los parquet

In [3]:
df_california = pd.read_parquet('gs://data_cruda/Google/California.parquet', storage_options={'token': 'cloud'})
df_florida = pd.read_parquet('gs://data_cruda/Google/Florida.parquet', storage_options={'token': 'cloud'})
df_pennsylvania = pd.read_parquet('gs://data_cruda/Google/Pennsylvania.parquet', storage_options={'token': 'cloud'})

### Transformaciones y creación del df_reviews a partir de los 3 estados

In [4]:
df_california['state'] = 'CA'
df_florida['state'] = 'FL'
df_pennsylvania['state'] = 'PA'

In [5]:
columnas_a_eliminar = ['pics', 'resp']
df_california.drop(columnas_a_eliminar, axis=1, inplace=True)
df_florida.drop(columnas_a_eliminar, axis=1, inplace=True)
df_pennsylvania.drop(columnas_a_eliminar, axis=1, inplace=True)


In [6]:
df_california['time'] = pd.to_datetime(df_california['time'], unit='ms').dt.date
df_florida['time'] = pd.to_datetime(df_florida['time'], unit='ms').dt.date
df_pennsylvania['time'] = pd.to_datetime(df_pennsylvania['time'], unit='ms').dt.date


In [7]:
df_reviews = pd.concat([df_california, df_florida, df_pennsylvania], ignore_index=True)

#### Achico la data

In [8]:
df_reviews['time'] = pd.to_datetime(df_reviews['time'])

In [9]:
df_reviews = df_reviews[df_reviews['time'].dt.year >= 2018]

In [10]:
conteo_por_gmap_id = df_reviews.groupby('gmap_id').size()
gmap_id_con_mas_de_1000_registros = conteo_por_gmap_id[conteo_por_gmap_id > 1000].index
df_reviews = df_reviews[df_reviews['gmap_id'].isin(gmap_id_con_mas_de_1000_registros)]

In [11]:
df_reviews.drop_duplicates(inplace=True)

### df_dates

In [12]:
df_dates = pd.DataFrame({'date': df_reviews['time'].unique()})

In [13]:
df_dates = df_dates.sort_values(by='date')

In [14]:
df_dates.insert(0, 'id_date', range(1, len(df_dates) + 1))

In [15]:
df_dates = df_dates.reset_index(drop=True)

In [16]:
# df_dates.to_parquet('gs://data_limpia/Google/Google_dates.parquet', compression='gzip')

In [17]:
df_reviews = df_reviews.merge(df_dates[['date', 'id_date']], left_on='time', right_on='date', how='left')

In [18]:
df_reviews.drop(['date', 'time'], axis=1, inplace=True)

#### Saco los registros de junio del 2021

In [19]:
boolean_mask_junio_2021 = (df_dates['date'].dt.year == 2021) & (df_dates['date'].dt.month == 6)
id_dates_a_eliminar = df_dates.loc[boolean_mask_junio_2021, 'id_date'].tolist()
df_reviews = df_reviews[~df_reviews['id_date'].isin(id_dates_a_eliminar)]

In [20]:
df_dates = df_dates[~boolean_mask_junio_2021]

### Creación de df_users

In [21]:
df_users = df_reviews[['user_id', 'name']]

In [22]:
df_reviews.drop('name', axis=1, inplace=True)

In [23]:
df_users.drop_duplicates(inplace=True)

In [24]:
# df_users.to_parquet('gs://data_limpia/Google/Google_users.parquet', compression='gzip')

### Importación de la metadata

In [25]:
df_metadata_filtrada = pd.read_parquet('gs://data_cruda/Google/metadatafinal.parquet', storage_options={'token': 'cloud'})

In [26]:
df_metadata_filtrada.drop(['avg_rating', 'num_of_reviews', 'address', 'price', 'relative_results', 'url', 'state', 'hours', 'MISC'], axis=1, inplace=True)

In [27]:
df_metadata_filtrada = df_metadata_filtrada.drop_duplicates(subset='gmap_id', keep='first')

In [28]:
gmap_ids_en_reviews = df_reviews['gmap_id'].unique()
df_metadata_filtrada = df_metadata_filtrada[df_metadata_filtrada['gmap_id'].isin(gmap_ids_en_reviews)]


In [29]:
df_metadata_filtrada.reset_index(drop=True, inplace=True)

### Columna MISC

In [30]:
#misc = pd.DataFrame({'MISC': df_metadata_filtrada['MISC'].astype(str)})

In [31]:
#misc = misc.apply(lambda x: [item.replace("[[", "[").replace("(", "").replace("[array", "").replace("\n ", "").replace(" None", "").replace("None", "").replace("array", "").replace("dtype=object", "").replace(")", "").replace("]]", "]").replace("], ]", "]").replace("[ [", "[").strip() for item in x])

In [32]:
#misc = misc.apply(lambda x: [item.replace(", [", ",").replace("],", ",").replace("[", "").replace("]", "").replace("\n", "").strip() for item in x])

In [33]:
pd.set_option('display.max_colwidth', None)

In [34]:
#misc["MISC"] = misc["MISC"].apply(lambda x: x.split(','))
#misc.head(2)

In [35]:
#misc['gmap_id'] = df_metadata_filtrada['gmap_id']

In [36]:
#records = []

#for index, row in misc.iterrows():
    #gmap_id = row['gmap_id']
    #misc_list = row['MISC']
    #for item in misc_list:
        #records.append({'gmap_id': gmap_id, 'misc': item})

#df_link_misc = pd.DataFrame(records)

#df_link_misc.head(2)

In [37]:
#misc.reset_index(drop=True, inplace=True)

In [38]:
#MISC_2 = pd.DataFrame(misc["MISC"].explode().str.strip())
#MISC_2.head(2)

In [39]:
#MISC_2 = MISC_2.drop_duplicates(subset='MISC', keep='first')

In [40]:
#MISC_2.tail(15)

In [41]:
#MISC_2['id_misc'] = range(1, len(MISC_2) + 1)
#MISC_2.reset_index(drop=True, inplace=True)

In [42]:
#MISC_2.to_parquet('gs://data_clean/Google/misc.parquet', compression='gzip')

In [43]:
#df_link_misc['misc'] = df_link_misc['misc'].str.strip()
#MISC_2['MISC'] = MISC_2['MISC'].str.strip()

In [44]:
#df_link_misc = df_link_misc.merge(MISC_2[['MISC', 'id_misc']], left_on='misc', right_on='MISC', how='left')

In [45]:
#df_link_misc.drop('misc', axis=1, inplace=True)

In [46]:
#df_link_misc.tail(2)

In [47]:
#df_link_misc.to_parquet('gs://data_clean/Google/link_misc.parquet', compression='gzip')


In [48]:
#df_metadata_filtrada.drop('MISC', axis=1, inplace=True)

### Columna hours

In [49]:
#df_metadata_filtrada['id_hours'] = range(1, len(df_metadata_filtrada) + 1)


In [50]:
#tabla_hours = df_metadata_filtrada[['hours', 'id_hours']]

In [51]:
#tabla_hours.head(2)

In [52]:
#hours_complete = tabla_hours[tabla_hours['hours'].notnull()]

In [53]:
#hours_complete.shape[0]

In [54]:
# Crear una lista vacía para almacenar los resultados
#horarios = []

# Lista de días de la semana
#dias_semana = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Iterar sobre cada registro de horas en hours_complete['hours']
#for registros_horas, id_registro in zip(hours_complete['hours'], hours_complete['id_hours']):
    # Crear un diccionario vacío para almacenar los horarios del registro actual
    #horarios_registro = {}

    # Iterar sobre cada registro en el array de registros de horas
    #for registro in registros_horas:
        # Obtener el día de la semana y el horario
        #dia_semana = registro[0]
        #horario = registro[1]

        # Agregar el horario al diccionario de horarios del registro actual
        #horarios_registro[dia_semana] = horario

    # Agregar el diccionario de horarios del registro actual a la lista de horarios
    #horarios.append([id_registro] + [horarios_registro.get(dia, '') for dia in dias_semana])

# Crear un DataFrame a partir de la lista de horarios
#horarios_df = pd.DataFrame(horarios, columns=['id_hours'] + dias_semana)

# Imprimir el DataFrame de resultados
#horarios_df.head(4)

In [55]:
#tabla_hours = pd.merge(tabla_hours, horarios_df, on='id_hours', how = 'left')

In [56]:
#tabla_hours.drop('hours', axis = 1, inplace = True)

In [57]:
# Función para transformar los registros a formato deseado
#def transformar_horarios(horario):

    #if horario == "Open 24 hours" or horario == "Closed" or pd.isnull(horario):
      #return horario

    # Utilizar expresiones regulares para extraer las partes del horario
    #patron = r"(\d{1,2})(:)?(\d{2})?([APM]+)?\u2013?(\d{1,2})?(:)?(\d{2})?([APM]+)?"
    #coincidencia = re.match(patron, horario)

    #if coincidencia:
      #hora_inicio = coincidencia.group(1)
      #minutos_inicio = coincidencia.group(3)
      #am_pm_inicio = coincidencia.group(4)
      #hora_fin = coincidencia.group(5)
      #minutos_fin = coincidencia.group(7)
      #am_pm_fin = coincidencia.group(8)

      # Formatear la hora de inicio
      #if hora_inicio:
        #if len(hora_inicio) < 2:
          #hora_inicio = f"0{hora_inicio}"
        #if minutos_inicio:
          #if len(minutos_inicio) < 2:
            #minutos_inicio = f"{minutos_inicio}0"
        #else:
          #minutos_inicio = f"00"
        #if am_pm_inicio:
          #am_pm_inicio = am_pm_inicio.upper()
        #else:
          #if int(hora_inicio) == 12:
            #am_pm_inicio = f"PM"
          #else:
            #if am_pm_fin == "AM":
              #am_pm_inicio = f"AM"
            #else:
              #am_pm_inicio = f"PM"

        #hora_total_inicio = f"{hora_inicio}:{minutos_inicio}{am_pm_inicio}"
      #else:
        #hora_inicio = ""

      # Formatear la hora de fin
      #if hora_fin:
        #if len(hora_fin) < 2:
          #hora_fin = f"0{hora_fin}"
        #if minutos_fin:
          #if len(minutos_fin) < 2:
            #minutos_fin = f"{minutos_fin}0"
        #else:
          #minutos_fin = f"00"
        #if am_pm_fin:
          #am_pm_fin = am_pm_fin.upper()
        #else:
          #am_pm_fin = f"PM"

        #hora_total_fin = f"{hora_fin}:{minutos_fin}{am_pm_fin}"
      #else:
        #hora_fin = ""

      # Formatear el horario en el formato deseado
      #horario_transformado = f"{hora_total_inicio}\u2013{hora_total_fin}"
      #return horario_transformado

    #return horario

In [58]:
#tabla_hours[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']] = tabla_hours[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']].applymap(transformar_horarios)

In [59]:
#tabla_hours.fillna('')

In [61]:
# tabla_hours['description'] = (
#     'Monday ' + tabla_hours['Monday'] + ', ' +
#     'Tuesday ' + tabla_hours['Tuesday'] + ', ' +
#     'Wednesday ' + tabla_hours['Wednesday'] + ', ' +
#     'Thursday ' + tabla_hours['Thursday'] + ', ' +
#     'Friday ' + tabla_hours['Friday'] + ', ' +
#     'Saturday ' + tabla_hours['Saturday'] + ', ' +
#     'Sunday ' + tabla_hours['Sunday']
# )

In [ ]:
# tabla_hours = tabla_hours.drop(columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], axis=1)

In [ ]:
# df_metadata_filtrada.drop('hours', axis=1, inplace=True)

In [ ]:
# tabla_hours.rename(columns={'description': 'schedule'}, inplace=True)

In [ ]:
# df_metadata_filtrada = df_metadata_filtrada.merge(tabla_hours[['id_hours', 'schedule']], on='id_hours', how='left')

In [ ]:
# tabla_hours = tabla_hours.drop_duplicates(subset=['schedule'])

In [ ]:
# tabla_hours = tabla_hours.reset_index(drop=True)
# tabla_hours['id_hours'] = tabla_hours.index + 1

In [ ]:
# df_metadata_filtrada = df_metadata_filtrada.merge(tabla_hours[['id_hours', 'schedule']], on='schedule', how='left')

In [ ]:
# df_metadata_filtrada.drop(['schedule', 'id_hours_x'], axis=1, inplace=True)

In [ ]:
# df_metadata_filtrada.rename(columns={'id_hours_y': 'id_hours'}, inplace=True)

In [ ]:
#tabla_hours.to_parquet('gs://data_clean/Google/schedule.parquet', compression='gzip')


### Columna category

In [62]:
df_metadata_category = df_metadata_filtrada[['gmap_id', 'category']]

In [63]:
# Convertir la lista de categorías en cadenas
df_metadata_category['category'] = df_metadata_category['category'].apply(', '.join)

# Dividir las categorías en columnas separadas
categories_df = df_metadata_category['category'].str.split(', ', expand=True)

# Apilar las columnas en una sola columna 'description'
categories_stacked = categories_df.stack().reset_index(level=1, drop=True).rename('description')

# Crear un DataFrame con descripciones únicas y asignar IDs
df_categories = pd.DataFrame({'description': categories_stacked.unique()}).reset_index(drop=True)
df_categories['id_cat'] = df_categories.index + 1

# Mostrar el nuevo DataFrame con IDs asignados a las categorías
df_categories.head()


,description,id_cat
0,Department store,1
1,Clothing store,2
2,Craft store,3
3,Electronics store,4
4,Furniture store,5


In [64]:
# Dividir las categorías en columnas separadas en df_metadata_category
categories_df = df_metadata_category['category'].str.split(', ', expand=True)

# Reorganizar el índice para mantener 'gmap_id'
categories_df['gmap_id'] = df_metadata_category['gmap_id']
categories_df = categories_df.set_index('gmap_id')

# Apilar las columnas en una sola columna 'description' y resetear el índice
categories_stacked = categories_df.stack().reset_index(level=1, drop=True).rename('description')

# Asignar 'id_cat' a cada 'description' en df_categories
categories_mapping = df_categories.set_index('description')['id_cat']

# Crear 'id_cat' en categories_stacked usando el mapeo
categories_stacked = categories_stacked.map(categories_mapping)

# Crear df_link_cat a partir de la serie resultante
df_link_cat = pd.DataFrame({'id_cat': categories_stacked}).reset_index()

# Mostrar el nuevo DataFrame df_link_cat
df_link_cat.tail()


,gmap_id,id_cat
2448,0x80dc755aa2b28be1:0x51e22e4c135aeec9,279
2449,0x80dc755aa2b28be1:0x51e22e4c135aeec9,44
2450,0x80dc755aa2b28be1:0x51e22e4c135aeec9,46
2451,0x80dc755aa2b28be1:0x51e22e4c135aeec9,280
2452,0x80dc755aa2b28be1:0x51e22e4c135aeec9,47


In [65]:
df_metadata_filtrada.drop('category', axis=1, inplace=True)

In [66]:
# df_categories.to_parquet('gs://data_limpia/Google/Google_categories.parquet', compression='gzip')
# df_link_cat.to_parquet('gs://data_limpia/Google/Google_link_cat.parquet', compression='gzip')

### Exporto ```businesses``` y ```reviews```

In [67]:
# df_metadata_filtrada.to_parquet('gs://data_limpia/Google/Google_bussines.parquet', compression='gzip')

In [68]:
df_reviews = df_reviews[['gmap_id', 'user_id', 'rating', 'text', 'id_date' ,'state']]

In [69]:
# df_reviews.to_parquet('gs://data_limpia/Google/Google_reviews.parquet', compression='gzip')

In [74]:
df_users[df_users['name'] == 'A']

,user_id,name
6619,1.084746e+20,A
6688,1.042561e+20,A
33098,1.110277e+20,A
122559,1.159501e+20,A
168872,1.149099e+20,A
178610,1.076594e+20,A
203128,1.063958e+20,A
278280,1.151660e+20,A
313934,1.008034e+20,A
319118,1.124435e+20,A
